### QA 
1. input molecules graph data √
2. input correct env variable √  - btw: art is what seawater?
3. connect these data ×
只匹配环境变量 20×16 = 320 和 1023对不上（这个因为同一环境条件下对一个分子不止做了一次实验，明天再研究下能怎么匹配），我明天再改下，
还是得从表格取env_df
卡住

### 1. Load molecules graph datas from pickle

In [1]:
## Load the molecular data (.pkl) for molecules with Targeted BDE & BDFE 

import os
import pickle
import networkx as nx
from torch_geometric.utils import from_networkx

folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths in the folder
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# Load all graphs
graph_list = []
for file_path in pkl_files:
    with open(file_path, "rb") as file:
        graph = pickle.load(file)  # Load each .pkl graph
        graph_list.append(graph)  # Store in a list

print(f"Loaded {len(graph_list)} graphs")


Loaded 20 graphs


In [2]:
## 检查
import pandas as pd

# Select the first graph for visualization
nx_graph = graph_list[0]

# Convert node data to DataFrame
node_data = pd.DataFrame.from_dict(dict(nx_graph.nodes(data=True)), orient="index")

# Convert edge data to DataFrame
edge_data = pd.DataFrame.from_records([(u, v, d) for u, v, d in nx_graph.edges(data=True)], 
                                      columns=["source", "target", "attributes"])

# Expand edge attributes
if not edge_data.empty and isinstance(edge_data.iloc[0]["attributes"], dict):
    attr_df = edge_data["attributes"].apply(pd.Series)  # Convert dict to columns
    edge_data = pd.concat([edge_data.drop(columns=["attributes"]), attr_df], axis=1)

# Display DataFrames
from IPython.display import display
display(node_data)
display(edge_data)



,symbol,rdkit_idx
0,C,0.0
1,C,1.0
2,C,2.0
3,C,3.0
4,C,4.0
5,C,5.0
6,C,6.0
7,C,7.0
8,C,8.0
9,C,9.0


,source,target,bond_index,bde_pred,bdfe_pred
0,0,1,0.0,89.382645,75.711845
1,0,H_23,25.0,100.077171,91.049126
2,1,2,1.0,85.872452,71.412849
3,1,H_27,29.0,97.163109,87.689636
4,2,3,2.0,85.041306,70.000282
5,2,H_28,30.0,95.392189,86.257263
6,3,4,3.0,83.115479,66.995277
7,3,H_30,32.0,94.518471,84.748619
8,4,5,NaN,NaN,NaN
9,4,10,NaN,NaN,NaN


Thinking: 为什么rdix里H对应的都是NAN 我们转化时用的是H-H 单H 键不需要 再进入rdix编号

备注：os.listdir... 列出文件夹中文件名称，迭代获取，保留.pkl,output list
     "rb"对口pkl二进制

In [3]:
## Convert Graphs to PyTorch Geometric Format for GNN model
# 方便训练和GNN计算，转化为PyG

import os
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data
import networkx as nx

# Path to dataset folder (Update this)
folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# List to store PyTorch Geometric graphs
pytorch_graphs = []

# **Function to handle missing hydrogen atom labels**
def clean_node_labels(node_data):
    """Replace NaN hydrogen labels with unique identifiers."""
    return {node: idx if not pd.isna(node) else f"H_{idx}" for idx, node in enumerate(node_data)}

# 分别提取节点和边的数据重构graph
# **Loop through each .pkl file and convert to PyG format**
for i, file_path in enumerate(pkl_files):
    with open(file_path, "rb") as file:
        nx_graph = pickle.load(file)  # Load NetworkX graph

        if isinstance(nx_graph, nx.Graph):
            # **Relabel nodes to avoid NaN values**
            mapping = clean_node_labels(nx_graph.nodes())
            nx_graph = nx.relabel_nodes(nx_graph, mapping)

            # **Convert Edges to PyG format (edge_index)**
            edge_index = torch.tensor(list(nx_graph.edges()), dtype=torch.long).T
            if edge_index.numel() == 0:  # Handle cases with no edges
                edge_index = torch.empty((2, 0), dtype=torch.long)

            # **Extract Node Features (Use Atomic Symbols)**
            first_node = next(iter(nx_graph.nodes), None)
            if first_node is not None and "symbol" in nx_graph.nodes[first_node]:
                node_features = torch.tensor(
                    [[1 if nx_graph.nodes[n]["symbol"] == "C" else 0] for n in nx_graph.nodes()],
                    dtype=torch.float
                )  # One-hot encoding Carbon (C) vs Hydrogen (H)
            else:
                node_features = torch.zeros((len(nx_graph.nodes()), 1), dtype=torch.float)  # Default: 1D zero feature

            # **Extract Edge Features (BDE, BDFE) - Handle NoneType**
            edge_features_list = []
            for u, v in nx_graph.edges():
                bde = nx_graph[u][v].get("bde_pred", 0.0)  # Replace None with 0.0
                bdfe = nx_graph[u][v].get("bdfe_pred", 0.0)  # Replace None with 0.0
                edge_features_list.append([bde if bde is not None else 0.0, bdfe if bdfe is not None else 0.0])

            edge_features = torch.tensor(edge_features_list, dtype=torch.float) if edge_features_list else None

            # **Create PyG Graph**
            pyg_graph = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

            # **Store Processed Graph**
            pytorch_graphs.append(pyg_graph)
            print(f"Converted Graph {i+1}: {len(nx_graph.nodes)} nodes, {len(nx_graph.edges)} edges")

print(f"\n Total converted graphs: {len(pytorch_graphs)}")


Converted Graph 1: 42 nodes, 44 edges
Converted Graph 2: 43 nodes, 45 edges
Converted Graph 3: 52 nodes, 56 edges
Converted Graph 4: 54 nodes, 58 edges
Converted Graph 5: 56 nodes, 60 edges
Converted Graph 6: 56 nodes, 60 edges
Converted Graph 7: 58 nodes, 62 edges
Converted Graph 8: 58 nodes, 62 edges
Converted Graph 9: 60 nodes, 64 edges
Converted Graph 10: 60 nodes, 64 edges
Converted Graph 11: 62 nodes, 66 edges
Converted Graph 12: 62 nodes, 66 edges
Converted Graph 13: 46 nodes, 48 edges
Converted Graph 14: 48 nodes, 50 edges
Converted Graph 15: 52 nodes, 54 edges
Converted Graph 16: 52 nodes, 54 edges
Converted Graph 17: 53 nodes, 55 edges
Converted Graph 18: 53 nodes, 55 edges
Converted Graph 19: 48 nodes, 52 edges
Converted Graph 20: 48 nodes, 52 edges

 Total converted graphs: 20


In [10]:
print(pytorch_graphs[15])
print(pytorch_graphs[15].x)


Data(x=[52, 1], edge_index=[2, 54], edge_attr=[54, 2])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])


---

## 2. Load the environmental variable

In [ ]:
import pandas as pd

# Load the environmental data from Excel
env_file = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\dataset02.xlsx"
env_df = pd.read_excel(env_file, engine='openpyxl')

# Select only the relevant columns for the environment
env_columns = ["temperature", "seawater", "time", "concentration", "degradation_rate"]

# Ensure all columns exist in the dataset
env_df = env_df[env_columns].copy()

# Convert categorical "seawater" to numerical (if needed)
env_df["seawater"] = env_df["seawater"].map({"sea": 1, "art": 0})  # Map "sea" → 1, "art" → 0

# Drop rows with missing values
env_df = env_df.dropna().reset_index(drop=True)

# Check if it matches the number of graphs
print(f"Loaded {len(env_df)} environment rows")
print(env_df.head())


Loaded 1023 environment rows
   temperature  seawater  time  concentration  degradation_rate
0         35.6         1    30             70          0.670914
1         35.6         1    30             70          0.680071
2         35.6         1    30             70          0.655230
3         35.6         1    30             70          0.625193
4         35.6         1    30             70          0.605853


In [6]:
print(len(pytorch_graphs))  # Should be 20
print(len(env_df))          # Should also be 20

# If they don’t match, fix by truncating or aligning:
if len(env_df) > len(pytorch_graphs):
    env_df = env_df.iloc[:len(pytorch_graphs)].reset_index(drop=True)
elif len(env_df) < len(pytorch_graphs):
    print("Warning: Not enough environment rows, some graphs will be missing data.")


20
1023


---

## 3. Use environmental variables as Transformer Positional Encoding connect

simply use nx graph is enough

### 3.1  建立Build DataLoader
PYG的Data对象代表图形，图形在节点和边缘的数量上有所不同。默认的DataLoader试图像张量一样堆叠它们，是不行滴。
 Custom Collate？

In [ ]:
Mgraph = pytorch_graphs[i]
my_x = Mgraph.x

print(len(my_x))

48


In [ ]:
from torch_geometric.data import Data


# Suppose you have:
#  - x, edge_index, edge_attr for the molecule
#  - env_feats: your 4 environment variables (temperature, seawater, time, concentration)
#  - y: the degradation rate (our target)

graph_data = Data(
    x=my_x,
    edge_index=edge_index,
    edge_attr=edge_attr,
    env=torch.tensor(env_feats, dtype=torch.float),  # shape [4]
    y=torch.tensor([y], dtype=torch.float)           # shape [1], a regression target
)


NameError: name 'x' is not defined

In [17]:
import torch
from torch_geometric.data import Data

# We assume you already have:
#   1) pytorch_graphs = [G1, G2, ..., G20]
#      Each G is a PyG Data object with at least x, edge_index, edge_attr
#   2) env_df is a DataFrame with columns like:
#       ["temperature", "seawater", "time", "concentration", "degradation_rate"]
#      potentially 1,023 rows or more

new_graphs = []

# Outer loop: go through each of your 20 PyG graphs
for i, Mgraph in enumerate(pytorch_graphs):
    # Extract the graph info we need
    base_x = Mgraph.x
    base_edge_index = Mgraph.edge_index
    base_edge_attr = Mgraph.edge_attr

    # Inner loop: go through *all* environment rows
    # so each environment row gets paired with this graph
    for j, row in env_df.iterrows():
        # row["temperature"], row["seawater"], row["time"], row["concentration"]
        env_feats = [
            row["temperature"],
            row["seawater"],
            row["time"],
            row["concentration"],
        ]

        # The degradation rate is our regression target
        y_value = row["degradation_rate"]

        # Create a new PyG Data object that merges:
        #  - the same graph (x, edge_index, edge_attr)
        #  - environment features (env)
        #  - target (y)
        graph_data = Data(
            x=base_x,
            edge_index=base_edge_index,
            edge_attr=base_edge_attr,
            env=torch.tensor(env_feats, dtype=torch.float),
            y=torch.tensor([y_value], dtype=torch.float)  # shape [1]
        )

        new_graphs.append(graph_data)

print("Created", len(new_graphs), "graphs with environment + target info.")


Created 0 graphs with environment + target info.


In [18]:
print("Before loops, len(pytorch_graphs) =", len(pytorch_graphs))
print("Before loops, len(env_df) =", len(env_df))

new_graphs = []

for i, Mgraph in enumerate(pytorch_graphs):
    print(f"Graph {i}: x shape = {Mgraph.x.shape}, edge_index shape = {Mgraph.edge_index.shape}")

    for j, row in env_df.iterrows():
        print(f"  env row {j} -> temperature: {row['temperature']}, time: {row['time']}, degrade: {row['degradation_rate']}")
        
        env_feats = [
            row["temperature"],
            row["seawater"],
            row["time"],
            row["concentration"]
        ]
        y_value = row["degradation_rate"]
        
        graph_data = Data(
            x=Mgraph.x,
            edge_index=Mgraph.edge_index,
            edge_attr=Mgraph.edge_attr,
            env=torch.tensor(env_feats, dtype=torch.float),
            y=torch.tensor([y_value], dtype=torch.float)
        )
        new_graphs.append(graph_data)

print("Created", len(new_graphs), "graphs with environment + target info.")


Before loops, len(pytorch_graphs) = 0
Before loops, len(env_df) = 20
Created 0 graphs with environment + target info.


### 3.2 建立 env + molecules graph层 → 建立GNN
由于PYG将多个图批量分为一个大图结构，因此我们需要data.batch来确保每个节点都会收到其相应图的正确环境嵌入
1.PYG将多个图形组合在一起→来自不同图形的节点被连接
2.data.batch tracks nodes属于哪个图。
3.我们使用env_pos_emb[data.batch]添加每个节点的正确环境效果
4.这确保每个节点都会获取其相应的环境编码

### 3.3 MSE/RMSE/R2 检验 → 测试model

---

In [12]:
# Suppose your environment DataFrame is called `df_env`.
# Suppose it has columns:
#  ["temperature", "seawater", "time", "concentration", "degradation_rate"]
# and each row matches one molecule in `pytorch_graphs`.

# 1. Create dataset
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=df_env)

# 2. Create dataloader
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate)

# 3. Initialize model
num_node_features = 1  # if your x is [N,1], e.g. one-hot for C vs H
env_input_dim = 4       # [temperature, concentration, time, maybe 'seawater']
model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# 4. Train
train_model(model, dataloader, device='cpu', epochs=20)


NameError: name 'df_env' is not defined

In [11]:
train_model(model, dataloader, epochs=20)

NameError: name 'model' is not defined

## 4. fixed code

In [13]:
print(len(pytorch_graphs))   # Should be 20
print(len(env_df))           # Must also be 20


20


NameError: name 'env_df' is not defined

In [18]:
import torch
from torch_geometric.data import Data, Dataset
from torch.utils.data import DataLoader

class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df):
        """
        Args:
            pyg_graphs: list of PyG Data objects (molecules)
            env_df: pandas DataFrame with environmental conditions & target
        """
        self.pyg_graphs = pyg_graphs
        self.env_df = env_df
        
        # 1. Expand molecule data to match env_df length
        self.graphs_expanded = [pyg_graphs[i % len(pyg_graphs)] for i in range(len(env_df))]

        # 2. Extract environmental conditions
        self.env_features = torch.tensor(
            self.env_df[["temperature", "seawater", "time", "concentration"]].values, dtype=torch.float
        )

        # 3. Extract degradation rate as target
        self.targets = torch.tensor(self.env_df["degradation_rate"].values, dtype=torch.float)

    def __len__(self):
        return len(self.env_df)  # Matches number of experimental conditions

    def __getitem__(self, idx):
        graph = self.graphs_expanded[idx]  # Get corresponding molecule
        env_features = self.env_features[idx]  # Get environmental variables
        target = self.targets[idx]  # Get degradation rate

        return graph, env_features, target


In [19]:
# 创建 Dataset
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=env_df)

# 创建 DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 检查 Dataset 结构
graph, env_features, target = dataset[0]
print(graph)  # Should be a PyG Data object
print(env_features)  # Should be a tensor with 4 environmental variables
print(target)  # Should be a single float value


Data(x=[42, 1], edge_index=[2, 44], edge_attr=[44, 2])
tensor([35.6000,  1.0000, 30.0000, 70.0000])
tensor(0.6709)


In [20]:
# 假设你的 GNN 结构如下：
num_node_features = 1  # 节点特征
env_input_dim = 4  # 环境变量：[温度，海水类型，时间，浓度]
model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# 训练模型
train_model(model, dataloader, device='cpu', epochs=20)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

---

In [13]:
import itertools
import torch
import pandas as pd

# 所有环境变量的可能取值
temperature_values = [35.6, 86.0]
seawater_values = [0, 1]  # 这里用 0 (sea), 1 (art) 代替字符串
time_values = [30, 60]
concentration_values = [70, 700]

# **创建所有组合（16个环境变量组合）**
env_conditions = list(itertools.product(
    temperature_values, seawater_values, time_values, concentration_values
))

# 转为 Pandas DataFrame，便于查看
env_df = pd.DataFrame(env_conditions, columns=["temperature", "seawater", "time", "concentration"])
print(env_df)


    temperature  seawater  time  concentration
0          35.6         0    30             70
1          35.6         0    30            700
2          35.6         0    60             70
3          35.6         0    60            700
4          35.6         1    30             70
5          35.6         1    30            700
6          35.6         1    60             70
7          35.6         1    60            700
8          86.0         0    30             70
9          86.0         0    30            700
10         86.0         0    60             70
11         86.0         0    60            700
12         86.0         1    30             70
13         86.0         1    30            700
14         86.0         1    60             70
15         86.0         1    60            700


In [14]:
from torch_geometric.data import Dataset, Data, Batch
from torch.utils.data import DataLoader

class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df):
        """
        Args:
            pyg_graphs: list of 20 PyG 分子图
            env_df: 16 组环境变量
        """
        self.pyg_graphs = pyg_graphs  # 20个分子
        self.env_df = env_df

        # **创建 (20 × 16) 组实验数据**
        self.dataset = [
            (self.pyg_graphs[i % len(self.pyg_graphs)], torch.tensor(env, dtype=torch.float))
            for i in range(len(self.pyg_graphs) * len(self.env_df))
            for env in self.env_df.values
        ]

    def __len__(self):
        return len(self.dataset)  # 320 组数据

    def __getitem__(self, idx):
        graph, env_features = self.dataset[idx]
        return graph, env_features

# **创建 Dataset**
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=env_df)


In [15]:
class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df, target_values):
        """
        Args:
            pyg_graphs: list of 20 PyG 分子图
            env_df: 16 组环境变量
            target_values: 320 组目标降解率 (每个分子在不同环境下的实验结果)
        """
        self.pyg_graphs = pyg_graphs
        self.env_df = env_df
        self.target_values = target_values  # 目标值 (320个)

        # **创建 (20 × 16) 组实验数据**
        self.dataset = [
            (self.pyg_graphs[i % len(self.pyg_graphs)],  # Graph
             torch.tensor(env, dtype=torch.float),       # 环境变量
             torch.tensor(self.target_values[i], dtype=torch.float))  # 目标值
            for i in range(len(self.pyg_graphs) * len(self.env_df))
            for env in self.env_df.values
        ]

    def __len__(self):
        return len(self.dataset)  # 320 组数据

    def __getitem__(self, idx):
        graph, env_features, target = self.dataset[idx]
        return graph, env_features, target


In [16]:
def my_collate(batch):
    graphs, env_features, targets = zip(*batch)  # 拆分 Graph, 环境变量, 目标值
    graphs_batch = Batch.from_data_list(graphs)  # PyG 批量处理
    env_features_batch = torch.stack(env_features)  # 转 Tensor
    targets_batch = torch.stack(targets)  # 转 Tensor
    return graphs_batch, env_features_batch, targets_batch

# **创建 DataLoader**
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate)

# **检查 DataLoader**
graphs_batch, env_features_batch, targets_batch = next(iter(dataloader))
print(graphs_batch)  # PyG DataBatch
print(env_features_batch.shape)  # (batch_size, 4)
print(targets_batch.shape)  # (batch_size,)


ValueError: not enough values to unpack (expected 3, got 2)

In [17]:
# 假设 GNN 结构如下：
num_node_features = 1  # 节点特征
env_input_dim = 4  # 环境变量：[温度，海水类型，时间，浓度]

model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# **训练模型**
train_model(model, dataloader, device='cpu', epochs=20)


ValueError: not enough values to unpack (expected 3, got 2)